In [23]:
#Importing Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

from sklearn.metrics import classification_report
import numpy as np


In [24]:
#converting into tensors
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [25]:
#extracting zipfile
import zipfile

zip_path = "/content/animal_data.zip"
extract_path = "/content"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)



In [26]:
data_dir = "/content/animal_data"

dataset = datasets.ImageFolder(root=data_dir, transform=transform)

num_classes = len(dataset.classes)
print("Classes:", dataset.classes)


Classes: ['Bear', 'Bird', 'Cat', 'Cow', 'Deer', 'Dog', 'Dolphin', 'Elephant', 'Giraffe', 'Horse', 'Kangaroo', 'Lion', 'Panda', 'Tiger', 'Zebra']


In [27]:
#Training
train_size = int(0.6 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [28]:
#Model1

class BaselineCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 54 * 54, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x


In [29]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def train_model(model, train_loader, val_loader, epochs=5):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.to(device)

    for epoch in range(epochs):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{epochs} completed")

    return model


In [30]:
def evaluate_model(model, loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()

            y_pred.extend(preds)
            y_true.extend(labels.numpy())

    print(classification_report(y_true, y_pred, target_names=dataset.classes))


In [31]:
#Testing Model1
model1 = BaselineCNN(num_classes)
model1 = train_model(model1, train_loader, val_loader, epochs=5)
evaluate_model(model1, val_loader)


Epoch 1/5 completed
Epoch 2/5 completed
Epoch 3/5 completed
Epoch 4/5 completed
Epoch 5/5 completed
              precision    recall  f1-score   support

        Bear       0.36      0.33      0.35        39
        Bird       0.31      0.64      0.42        55
         Cat       0.40      0.42      0.41        50
         Cow       0.51      0.50      0.51        42
        Deer       1.00      0.24      0.39        50
         Dog       0.69      0.43      0.53        51
     Dolphin       0.76      0.87      0.81        62
    Elephant       0.76      0.32      0.45        59
     Giraffe       0.68      0.47      0.56        53
       Horse       0.22      0.71      0.34        48
    Kangaroo       0.32      0.41      0.36        64
        Lion       0.65      0.37      0.47        60
       Panda       1.00      0.35      0.52        51
       Tiger       0.86      0.44      0.58        43
       Zebra       0.86      0.82      0.84        51

    accuracy                      

In [32]:
#Model 2
class DeepCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3), nn.ReLU(), nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 26 * 26, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [ ]:
#training
model2 = DeepCNN(num_classes)
model2 = train_model(model2, train_loader, val_loader, epochs=5)
evaluate_model(model2, val_loader)


Epoch 1/5 completed
Epoch 2/5 completed
Epoch 3/5 completed
Epoch 4/5 completed
Epoch 5/5 completed


In [ ]:
#Model 3
model3 = models.resnet18(pretrained=True)

for param in model3.parameters():
    param.requires_grad = False

model3.fc = nn.Linear(model3.fc.in_features, num_classes)
model3.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model3.fc.parameters(), lr=0.001)

for epoch in range(5):
    model3.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model3(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/5 completed")


In [ ]:

evaluate_model(model3, val_loader)
